In [68]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import *
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import numpy as np

In [69]:
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
BATCH_SIZE = 16
EPOCHS = 3

LEARNING_RATE = 0.0008

In [70]:
transform = transforms.Compose([transforms.Resize([28, 28]),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

In [71]:
dataset = torchvision.datasets.ImageFolder(root = "../data/mnist-varres/train/",
                                          transform=transform)

train_set, validation_set = torch.utils.data.random_split(dataset, [TRAIN_SIZE, VALIDATION_SIZE])

train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers = 2)

validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size = BATCH_SIZE,
                                                shuffle = True,
                                                num_workers = 2)

In [72]:
test_set = torchvision.datasets.ImageFolder(root = "../data/mnist-varres/test/",
                                          transform=transform)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size = BATCH_SIZE,
                                          shuffle = True,
                                          num_workers = 2)

In [73]:
log_interval = 100

In [74]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1) # size: batch, 1, 28, 28
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1) # (batch, 16, 14, 14)
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1) # (batch, 32, 7, 7)
        self.fc1 = nn.Linear(64*3*3, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # (batch, 16, 28, 28) # TODO klopt het dat relu en maxpool zijn omgedraaid?
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # (batch, 32, 14, 14)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # (batch, 64, 7, 7)
        x = torch.flatten(x, 1) # (batch, 64, 3, 3)
        x = self.fc1(x)
        return F.log_softmax(x, dim = 1)

In [75]:
network = Net()
optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)

In [76]:
train_losses = []
train_counter = []
val_losses = []
val_counter = [i*TRAIN_SIZE for i in range(EPOCHS + 1)]
val_acc = []

In [77]:
# TODO Make sure to have "results" directory in "CNN/"
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), '../results/model_Q8.pth')
            torch.save(optimizer.state_dict(), '../results/optimizer_Q8.pth')

In [78]:
def test():
    network.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            output = network(data)
            val_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        val_loss /= VALIDATION_SIZE
        val_losses.append(val_loss)
        val_acc.append(float(100. * correct / VALIDATION_SIZE))
        print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            val_loss, correct, VALIDATION_SIZE,
            100. * correct / VALIDATION_SIZE))

# Training and validation 

In [79]:
for epoch in range(1, EPOCHS + 1):
    print(f'EPOCH: {epoch}')
    train(epoch)
    test()


EPOCH: 1
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.312194
Train Epoch: 1 [1600/50000 (3%)]	Loss: 1.845459
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1.091698
Train Epoch: 1 [4800/50000 (10%)]	Loss: 0.747624
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.180781
Train Epoch: 1 [8000/50000 (16%)]	Loss: 0.848353
Train Epoch: 1 [9600/50000 (19%)]	Loss: 1.051423
Train Epoch: 1 [11200/50000 (22%)]	Loss: 0.472286
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.268537
Train Epoch: 1 [14400/50000 (29%)]	Loss: 0.402681
Train Epoch: 1 [16000/50000 (32%)]	Loss: 0.421293
Train Epoch: 1 [17600/50000 (35%)]	Loss: 0.192027
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.534002
Train Epoch: 1 [20800/50000 (42%)]	Loss: 0.919927
Train Epoch: 1 [22400/50000 (45%)]	Loss: 0.164797
Train Epoch: 1 [24000/50000 (48%)]	Loss: 0.229819
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.108050
Train Epoch: 1 [27200/50000 (54%)]	Loss: 0.222339
Train Epoch: 1 [28800/50000 (58%)]	Loss: 0.550889
Train Epoch: 1 [30400/50000 (61%)]	Loss: 0.090447
Trai

In [80]:
# Store values
np.savetxt("../results/training_stats_Q17fixed.csv", [p for p in zip(train_counter, train_losses)], delimiter=',', fmt='%s')
np.savetxt("../results/validation_stats_Q17fixed.csv", [p for p in zip(val_losses, val_acc)], delimiter=',', fmt='%s')

# Performance on test set

In [81]:
def test_set_performance(test_loader, TEST_SIZE): 
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= TEST_SIZE
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, TEST_SIZE,
            100. * correct / TEST_SIZE))
    return test_loss, float(100 * correct / TEST_SIZE)

    

/Users/jannekehulsen/miniconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3030, Accuracy: 1147/10000 (11%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.295086
Train Epoch: 1 [1600/50000 (3%)]	Loss: 2.030319
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1.244026
Train Epoch: 1 [4800/50000 (10%)]	Loss: 1.067634
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.952617
Train Epoch: 1 [8000/50000 (16%)]	Loss: 0.579303
Train Epoch: 1 [9600/50000 (19%)]	Loss: 0.488854
Train Epoch: 1 [11200/50000 (22%)]	Loss: 0.791562
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.351586
Train Epoch: 1 [14400/50000 (29%)]	Loss: 0.549284
Train Epoch: 1 [16000/50000 (32%)]	Loss: 0.345628
Train Epoch: 1 [17600/50000 (35%)]	Loss: 0.133538
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.639107
Train Epoch: 1 [20800/50000 (42%)]	Loss: 0.137807
Train Epoch: 1 [22400/50000 (45%)]	Loss: 0.218945
Train Epoch: 1 [24000/50000 (48%)]	Loss: 0.195649
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.680345
Train Epoch: 1 [27200/50000 (54%)]	Loss: 0.480136
Train Epoch: 1 [28800/50000 (58%)]	Loss: 0.292999
Train

In [82]:
TEST_SIZE = 10000
final_test_avgloss, final_test_acc = test_set_performance(test_loader, TEST_SIZE)



Test set: Avg. loss: 0.1087, Accuracy: 9649/10000 (96%)



In [83]:
print('Final Test Accuracy = ', final_test_acc, '%')
print('Final Avg. Loss = ', final_test_avgloss)

Final Test Accuracy =  96.48999786376953 %
Final Avg. Loss =  0.10866481845663511


In [84]:
#Store values
np.savetxt("../results/test_stats_Q17fixed.csv", [(final_test_avgloss, final_test_acc)], delimiter=',', fmt='%s')

In [ ]:
# Store values
np.savetxt("../results/training_stats_Q17fixed.csv", [p for p in zip(train_counter, train_losses)], delimiter=',', fmt='%s')
np.savetxt("../results/validation_stats_Q17fixed.csv", [p for p in zip(val_losses, val_acc)], delimiter=',', fmt='%s')

NameError: name 'val_acc' is not defined